# Testing Processing For MIMIC

In [1]:
import json
import os

import pandas as pd

# Important for pandas bar progress
from tqdm import tqdm

tqdm.pandas()

# Test functions to check processing
import src.data_processing.MIMIC.test_functions as tests

In [2]:
with open("src/data_processing/MIMIC/MIMIC_PROCESSING_DEFAULT_VARS.json", "r") as f:
    DEFAULT_CONFIG = json.load(f)
    f.close()

    

In [3]:
assert os.path.exists(DEFAULT_CONFIG["SAVE_FD"] + "admissions_intermediate.csv")

In [41]:
vitals_S3 = (vitals_S2
       .assign(time_to_end=lambda x: x["outtime"] - x["charttime"])
       .sort_values(by=["stay_id", "time_to_end"], ascending=[True, False])
       .groupby("stay_id", as_index=False)
       .progress_apply(lambda x: _resample(x, time_feats = list(DEFAULT_CONFIG["VITALS_RENAMING_DIC"].values()), resampling_rule = DEFAULT_CONFIG["RESAMPLING_RULE"], resampling_col="time_to_end"))
       .reset_index(drop=True))
vitals_S3.to_csv(DEFAULT_CONFIG["SAVE_FD"] + "vitals_S3.csv", index=True, header=True)



100%|███████████████████████████████████████████████████████████████████████████████████████████| 28668/28668 [06:40<00:00, 71.59it/s]


In [81]:
from importlib import reload
reload(tests)

<module 'src.data_processing.MIMIC.test_functions' from '/home/hq-boss31/Projects/Github/VarPhenClustering/src/data_processing/MIMIC/test_functions.py'>

In [83]:
vitals_S3

,sampled_time_to_end,TEMP,HR,RR,SPO2,SBP,DBP,subject_id,stay_id,charttime,...,outtime_next,transfer_id,eventtype,careunit,intime,outtime,gender,age,ESI,deathtime
0,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,13340997,30000039,2165-10-06 13:10:00,...,NaT,30000039,ED,Emergency Department,2165-10-06 11:47:00,2165-10-06 20:18:00,1,80,3.0,NaT
1,0 days 01:00:00,98.05,90.0,19.0,96.5,153.5,80.0,13340997,30000039,2165-10-06 13:10:00,...,NaT,30000039,ED,Emergency Department,2165-10-06 11:47:00,2165-10-06 20:18:00,1,80,3.0,NaT
2,0 days 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,13340997,30000039,2165-10-06 13:10:00,...,NaT,30000039,ED,Emergency Department,2165-10-06 11:47:00,2165-10-06 20:18:00,1,80,3.0,NaT
3,0 days 03:00:00,NaN,96.0,22.0,98.0,165.0,76.0,13340997,30000039,2165-10-06 13:10:00,...,NaT,30000039,ED,Emergency Department,2165-10-06 11:47:00,2165-10-06 20:18:00,1,80,3.0,NaT
4,0 days 04:00:00,NaN,NaN,NaN,NaN,176.0,112.0,13340997,30000039,2165-10-06 13:10:00,...,NaT,30000039,ED,Emergency Department,2165-10-06 11:47:00,2165-10-06 20:18:00,1,80,3.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362201,0 days 18:00:00,NaN,74.0,16.0,100.0,122.0,75.0,17255106,39999939,2155-08-04 14:13:00,...,NaT,39999939,ED,Emergency Department,2155-08-04 11:15:00,2155-08-05 12:48:00,1,57,2.0,NaT
362202,0 days 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,17255106,39999939,2155-08-04 14:13:00,...,NaT,39999939,ED,Emergency Department,2155-08-04 11:15:00,2155-08-05 12:48:00,1,57,2.0,NaT
362203,0 days 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,17255106,39999939,2155-08-04 14:13:00,...,NaT,39999939,ED,Emergency Department,2155-08-04 11:15:00,2155-08-05 12:48:00,1,57,2.0,NaT
362204,0 days 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,17255106,39999939,2155-08-04 14:13:00,...,NaT,39999939,ED,Emergency Department,2155-08-04 11:15:00,2155-08-05 12:48:00,1,57,2.0,NaT


In [91]:

vitals_S4.to_csv(DEFAULT_CONFIG["SAVE_FD"] + "vitals_S4.csv", index=True, header=True)
print("====", end="\r")

In [96]:
test = vitals_S4.groupby("stay_id").progress_apply(lambda x: x.dropna(subset=["HR"]).sampled_time_to_end.min())

100%|████████████████████████████████████████████████████████████████████████████████████████████| 8694/8694 [00:49<00:00, 174.28it/s]


In [104]:

# Useful objects to compute
feats_to_check = list(DEFAULT_CONFIG["VITALS_RENAMING_DIC"].values())
resampling_rule = DEFAULT_CONFIG["RESAMPLING_RULE"]

# Useful timedelta defn
max_allowed_time_before_outtime = pd.Timedelta(hours = DEFAULT_CONFIG["LAST_OBVS_TIME_TO_EXIT"])

# Ensure the first seen observation is not too far away from end of stay. Resampling starts at 0, however, observations for time varying features
# will be missing until we see an observation. Therefore, we can "recover" the first observation by taking the minimum of the non-NA observations.
vitals_S5 = (
    vitals_S4
    .groupby("stay_id", as_index=True)
    .filter(lambda x: 
        x
        .dropna(subset = feats_to_check, how="all")
        .sampled_time_to_end
        .min()
            <=
        max_allowed_time_before_outtime
    )
)

# Test and save
tests.test_last_observation_within_window_to_outtime(vitals_S5, max_allowed_time_before_outtime)

vitals_S5.to_csv(DEFAULT_CONFIG["SAVE_FD"] + "vitals_S5.csv", index=True, header=True)
print("====", end="\r")


TypeError: test_last_observation_within_window_to_outtime() missing 1 required positional argument: 'time_feats'

In [107]:
reload(tests)
tests.test_vitals_processed_correctly(vitals_S5, DEFAULT_CONFIG)


Testing vitals were processed correctly and make sense.

Testing ids are complete for params ('subject_id', 'stay_id', 'sampled_time_to_end')
Test passed!

Testing next transfer information is consistent.
Test passed!

Testing stays have sufficient data based on info dic parameters.
Test passed!

Testing last observation is AT MOST td_window before admission outtime.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8364/8364 [00:47<00:00, 176.77it/s]


Test passed!

Testing resampling data is linear from min to max per patient.


100%|███████████████████████████████████████████████████████████████████████████████████████████| 8364/8364 [00:06<00:00, 1287.80it/s]

Test passed!
Test passed!


In [109]:

vitals_S5.to_csv(DEFAULT_CONFIG["SAVE_FD"] + "vitals_intermediate.csv", index=False, header=True)
print("=====", end="\r")


=====/r